Runtimes for increasing dataset
========

In [ ]:
import numpy as np
import pandas as pd
import glob, os, os.path, sys, warnings, math, time, re, collections

%matplotlib inline
import matplotlib.pyplot as plt
from plotting.plots import *

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from techniques import sortedTechniques
from metric_learn import LMNN, NCA, LFDA, Covariance, MetricEvolution, NeuralNetworkTransformer, FullMatrixTransformer, DiagonalMatrixTransformer
from metric_learn import ITML_Supervised, SDML_Supervised, LSML_Supervised, RCA_Supervised


datasetsDirectory = 'datasets'
resultsDirectory = 'results/runtimes-cont'

if not os.path.exists(resultsDirectory):
    os.makedirs(resultsDirectory)

# np.set_printoptions(precision=7, suppress=True, threshold=np.nan)
np.set_printoptions(formatter={'float': lambda x: "{0:0.5f}".format(x)})

import pickle
def save_obj(obj, name):
    with open('{}/{}.pkl'.format(resultsDirectory, name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open('{}/{}.pkl'.format(resultsDirectory, name), 'rb') as f:
        return pickle.load(f)
    
def exists_obj(name):
    return os.path.exists('{}/{}.pkl'.format(resultsDirectory, name))

def gfn(filename):
    odir = '../thesis-distance-metric-learning/thesis/graphs/learning-times'
    if not os.path.exists(odir):
        os.makedirs(odir)
    return '{}/{}'.format(odir, filename)

In [ ]:
common_ea_params = {
    's__stats': None,
    's__n_gen': 200,
    'transformer': 'full',
}

bestAlgorithms = [
    (['full'], 'stand+Cov+kNN', Covariance()),
    (['full'], 'stand+LMNN+kNN', LMNN(k=4, regularization=0.9, learn_rate=1e-8, max_iter=200)),
    (['full'], 'stand+NCA+kNN', NCA(max_iter=200, learning_rate=0.01)),
    (['full'], 'stand+LFDA+kNN', LFDA(k=3, num_dims=None)),
    
    (['full', 'diagonal'], 'stand+CMAES+kNN', MetricEvolution(**common_ea_params, strategy='cmaes', fitnesses=[('knn', {'n_neighbors':8})])),
    (['full', 'diagonal'], 'stand+CMAESFme+kNN', MetricEvolution(**common_ea_params, strategy='cmaes', fitnesses='wfme')),
    
    (['full', 'diagonal'], 'stand+JDE+kNN', MetricEvolution(**common_ea_params, strategy='jde', fitnesses='wfme')),
    (['full', 'diagonal'], 'stand+JDEkNN+kNN', MetricEvolution(**common_ea_params, strategy='jde', fitnesses=[('knn', {'n_neighbors':8})])),
]

mainAlgorithms = [
    (['full'], 'stand+Cov+kNN', Covariance()),
    (['full'], 'stand+LMNN+kNN', LMNN(k=4, regularization=0.9, learn_rate=1e-8, max_iter=200)),
    (['full'], 'stand+NCA+kNN', NCA(max_iter=200, learning_rate=0.01)),
    (['full'], 'stand+LFDA+kNN', LFDA(k=3, num_dims=None)),
    
    (['full', 'diagonal'], 'stand+CMAES+kNN', MetricEvolution(**common_ea_params, strategy='cmaes', fitnesses=[('knn', {'n_neighbors':8})])),
    (['full', 'diagonal'], 'stand+JDE+kNN', MetricEvolution(**common_ea_params, strategy='jde', fitnesses='wfme')),
    
    (['full', 'diagonal'], 'stand+CMAESfMe+kNN', MetricEvolution(**common_ea_params, strategy='cmaes', fitnesses='wfme')),
    (['full', 'diagonal'], 'stand+JDEkNN+kNN', MetricEvolution(**common_ea_params, strategy='jde', fitnesses=[('knn', {'n_neighbors':8})])),
]

In [ ]:
data = pd.read_csv('datasets/digits10.csv', sep=',', skiprows=1, header=0)
y = data['class']
X = data.drop(['class'], axis=1).values

le = LabelEncoder()
y = le.fit_transform(y)

for _ in range(10):
    for dim in range(2, 10):
        pca = PCA(n_components=dim)
        X_trans = pca.fit_transform(X)
        
        split_sizes_list = [500]
        if dim == 5:
            split_sizes_list = list(range(100, 1501, 100))
        
        for split_size in split_sizes_list:
            X_train, X_test, y_train, y_test = train_test_split(X_trans, y, train_size=split_size)

            imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=False)
            X_train = imputer.fit_transform(X_train)
            X_test = imputer.transform(X_test)

            standard = StandardScaler()
            X_train = standard.fit_transform(X_train)
            X_test = standard.transform(X_test)

            for transformers, methodName, method in mainAlgorithms:
                for transformer in transformers:
#                     if transformer=='diagonal': continue

                    dump_filename = '{}__{}__{}__{}'.format(methodName, transformer, dim, split_size)
                    if not exists_obj(dump_filename):
                        save_obj([], dump_filename)

                    prev_runtimes = load_obj(dump_filename)

                    if len(prev_runtimes)>=5:
                        print('skipping {}, already exists'.format(dump_filename))
                        continue

                    print("starting {} ({})".format(dump_filename, X_train.shape))
                    print("{} runs so far: {}".format(len(prev_runtimes), prev_runtimes))

                    try:
                        start = time.clock()

                        me = method
#                         me.params['transformer'] = transformer
                        me.fit(X_train, y_train)

                        end = time.clock()

                        save_obj(prev_runtimes+[end-start], dump_filename)
                    except:
                        print("Unexpected error:", sys.exc_info()[0])

In [ ]:
results = []
for file in glob.glob("{}/*.pkl".format(resultsDirectory)):
    results.append(file)
results.sort()

resultsByDataset = {}
for x in results:
    _,_,filename = re.split('/|\\\\', x)
    methodName,transformer,dims,samples = filename[:-4].split('__')
    samples = int(samples)
    dims = int(dims)
    
    if methodName not in resultsByDataset:
        resultsByDataset[methodName] = {}
    if transformer not in resultsByDataset[methodName]:
        resultsByDataset[methodName][transformer] = collections.OrderedDict()
    if samples not in resultsByDataset[methodName][transformer]:
        resultsByDataset[methodName][transformer][samples] = collections.OrderedDict()

    resultsByDataset[methodName][transformer][samples][dims] = load_obj(filename[:-4])

In [ ]:
# methodTitles = list(filter(lambda x:x[0][:5]=='stand', sortedTechniques))[-4:]
methodTitles = [
#     ('stand+Cov+kNN', 'Covariance'),
    ('stand+LMNN+kNN', 'LMNN'),
    ('stand+NCA+kNN', 'NCA'),
#     ('stand+LFDA+kNN', 'LFDA'),
    
    ('stand+CMAES+kNN', 'CMAES.kNN'),
    ('stand+JDEkNN+kNN', 'jDE.kNN'),

    ('stand+CMAESfMe+kNN', 'CMAES.fMe'),
    ('stand+JDE+kNN', 'jDE.fMe'),
]

transformerName = {
    'full': 'full',
    'diagonal': 'diag',
}

In [ ]:
# [np.mean(v) for k,v in resultsByDataset['stand+CMAES+kNN']['full'][500].items()]
# resultsByDataset['stand+CMAES+kNN']['full'][500]
# resultsByDataset['stand+CMAES+kNN']

In [ ]:
fig, axes = startGraphing('Learning times for an increasing dimension (fixed number of samples)'.format(), 1, 2, size=(8, 12), sharey=False)
alldata = resultsByDataset
data = []
titles = []
markers = []
for transformer in ['full', 'diagonal']:
    for method, title in methodTitles:

        if method not in alldata:
            print('{}'.format(method))
            continue
        if transformer not in alldata[method]:
            print('{} {}'.format(method, transformer))
            continue
        if 500 not in alldata[method][transformer]:
            print('{} {} {}'.format(method, transformer, 500))
            continue

        data.append(np.asarray([np.mean(v) for k,v in sorted(alldata[method][transformer][500].items())]))
#         print(data)
#         print(datasetName, title, math.ceil(data['runtime']))

        if '.' in title: title += " ({})".format(transformerName[transformer])
        titles.append(title)
        markers.append('-o' if transformer=='full' else ':s')
        
plotLines(axes[0], data, x_ticks=range(2, 10), title=None, ylabel='learning time (seconds)', xlabel='dimension', markers=markers)
plt.yscale('log')
plotLines(axes[1], data, x_ticks=range(2, 10), title=None, ylabel='learning time (seconds)', xlabel='dimension', markers=markers)
endGraphing(fig, legend=titles, filename=gfn('dimensions'), move_title=.94, adjust_legend=.12)
#     break

In [ ]:
fig, axes = startGraphing('Learning times for an increasing number of samples (fixed dimension)'.format(), 1, 2, size=(8, 12), sharey=False)
alldata = resultsByDataset
data = []
titles = []
for transformer in ['full', 'diagonal']:
    for method, title in methodTitles:

        if method not in alldata \
        or transformer not in alldata[method]:
            print('{} {}'.format(method, transformer))
            continue

        data.append(np.asarray([np.mean(v[5]) for k,v in sorted(alldata[method][transformer].items())]))
#         print(data)
#         print(datasetName, title, math.ceil(data['runtime']))

        if '.' in title: title += " ({})".format(transformerName[transformer])
        titles.append(title)
        
print(len(data))
plotLines(axes[0], data, x_ticks=range(100, 1501, 100), title=None, ylabel='learning time (seconds)', xlabel='number of samples', markers=markers)
plt.yscale('log')
plotLines(axes[1], data, x_ticks=range(100, 1501, 100), title=None, ylabel='learning time (seconds)', xlabel='number of samples', markers=markers)
endGraphing(fig, legend=titles, filename=gfn('samples'), move_title=.94, adjust_legend=.12)
#     break

In [ ]:
# ## RENAMING results for full/diag
# results = []
# for file in glob.glob("{}/*.pkl".format(resultsDirectory)):
#     results.append(file)
# results.sort()
# results

# for x in results:
#     os.rename(x, x.replace('.pkl', '__full.pkl'))